In [1]:
# ==================================================================================
# SANTA 2025 - 10 MINUTE AGGRESSIVE OPTIMIZER (RESTORED & FIXED)
# ==================================================================================

import os
import time
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from datetime import datetime, timedelta
from shutil import copy
from zipfile import ZipFile, ZIP_DEFLATED

# --- Libraries ---
from shapely import affinity
from shapely.geometry import Polygon, Point
from shapely.strtree import STRtree
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

# --- CONFIGURATION ---
MAX_MINUTES = 3         # Ровно 10 минут
GEOM_SCALE = 100000.0    # Масштаб для точности (НЕ МЕНЯТЬ)
FORCE_RESET = False      # False = продолжаем с 70.83. True = начинаем с нуля.

INPUT_CSV = '/kaggle/input/santa-2025-csv/santa-2025.csv'
SUBMISSION_CSV = 'submission.csv'

# Настройки точности
getcontext().prec = 50

# ==================================================================================
# 1. GEOMETRY CLASS
# ==================================================================================

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        # Размеры
        trunk_w = Decimal('0.15'); trunk_h = Decimal('0.2')
        base_w = Decimal('0.7'); mid_w = Decimal('0.4'); top_w = Decimal('0.25')
        tip_y = Decimal('0.8'); tier_1_y = Decimal('0.5'); tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0'); trunk_bottom_y = -trunk_h

        coords = [
            (0.0, float(tip_y)),
            (float(top_w/2), float(tier_1_y)), (float(top_w/4), float(tier_1_y)),
            (float(mid_w/2), float(tier_2_y)), (float(mid_w/4), float(tier_2_y)),
            (float(base_w/2), float(base_y)), (float(trunk_w/2), float(base_y)),
            (float(trunk_w/2), float(trunk_bottom_y)), (-float(trunk_w/2), float(trunk_bottom_y)),
            (-float(trunk_w/2), float(base_y)), (-float(base_w/2), float(base_y)),
            (-float(mid_w/4), float(tier_2_y)), (-float(mid_w/2), float(tier_2_y)),
            (-float(top_w/4), float(tier_1_y)), (-float(top_w/2), float(tier_1_y)),
        ]
        
        # Масштабируем координаты сразу
        self.scaled_coords = [(x * GEOM_SCALE, y * GEOM_SCALE) for x, y in coords]
        self.base_polygon = Polygon(self.scaled_coords)
        self.update_polygon()

    def update_polygon(self):
        rotated = affinity.rotate(self.base_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, 
            xoff=float(self.center_x) * GEOM_SCALE, 
            yoff=float(self.center_y) * GEOM_SCALE
        )

    def clone(self):
        return ChristmasTree(self.center_x, self.center_y, self.angle)

    def move(self, dx, dy):
        self.center_x += Decimal(dx)
        self.center_y += Decimal(dy)
        self.update_polygon()

# ==================================================================================
# 2. UTILS
# ==================================================================================

def parse_csv(csv_path) -> dict:
    df = pd.read_csv(csv_path)
    for c in ['x', 'y', 'deg']: df[c] = df[c].astype(str).str.strip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    return {gid: [ChristmasTree(r['x'], r['y'], r['deg']) for _, r in gdata.iterrows()] 
            for gid, gdata in df.groupby('group_id')}

def save_csv(dict_trees, filepath):
    data = []
    for gid in sorted(dict_trees.keys(), key=lambda x: int(x)):
        for i, t in enumerate(dict_trees[gid]):
            data.append({'id': f"{gid}_{i}", 'x': f"s{t.center_x}", 'y': f"s{t.center_y}", 'deg': f"s{t.angle}"})
    pd.DataFrame(data).to_csv(filepath, index=False)

def get_bounding_box_side(trees):
    if not trees: return Decimal(0)
    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = float('-inf'), float('-inf')
    for t in trees:
        b = t.polygon.bounds
        min_x, min_y = min(min_x, b[0]), min(min_y, b[1])
        max_x, max_y = max(max_x, b[2]), max(max_y, b[3])
    return Decimal(max(max_x - min_x, max_y - min_y)) / Decimal(GEOM_SCALE)

def get_score(trees, n):
    return (get_bounding_box_side(trees)**2) / Decimal(int(n))

def check_overlap(trees):
    if len(trees) < 2: return False
    polys = [t.polygon for t in trees]
    tree_index = STRtree(polys)
    for i, p in enumerate(polys):
        for idx in tree_index.query(p):
            if idx != i and p.intersects(polys[idx]) and not p.touches(polys[idx]):
                return True
    return False

# ==================================================================================
# 3. AGGRESSIVE OPTIMIZER
# ==================================================================================

def optimize_rotation_group(trees):
    """Поворачивает всю группу целиком (Global Rotation)."""
    pts = []
    for t in trees: pts.extend(t.polygon.exterior.coords)
    pts = np.array(pts)
    hull_pts = pts[ConvexHull(pts).vertices]
    
    def objective(a):
        r = np.radians(a)
        c, s = np.cos(r), np.sin(r)
        rot = hull_pts.dot(np.array([[c, s], [-s, c]]))
        return max(rot.max(axis=0) - rot.min(axis=0))

    res = minimize_scalar(objective, bounds=(0, 90), method='bounded')
    if abs(res.x) > 1e-5:
        mn, mx = pts.min(axis=0), pts.max(axis=0)
        cx, cy = (mn[0]+mx[0])/2, (mn[1]+mx[1])/2
        new_trees = []
        for t in trees:
            nt = t.clone()
            p = Point(float(nt.center_x)*GEOM_SCALE, float(nt.center_y)*GEOM_SCALE)
            p_rot = affinity.rotate(p, res.x, origin=(cx, cy))
            nt.center_x = Decimal(p_rot.x)/Decimal(GEOM_SCALE)
            nt.center_y = Decimal(p_rot.y)/Decimal(GEOM_SCALE)
            nt.angle += Decimal(res.x)
            nt.update_polygon()
            new_trees.append(nt)
        return new_trees
    return trees

def aggressive_shake(trees, iterations=200):
    """
    Агрессивная тряска:
    1. Вращает ОТДЕЛЬНЫЕ деревья.
    2. Двигает их к центру.
    """
    if len(trees) < 2: return trees
    
    best_trees = [t.clone() for t in trees]
    min_side = get_bounding_box_side(best_trees)
    
    # Центр (в реальных координатах)
    b = unary_union([t.polygon for t in trees]).bounds
    cx, cy = (b[0]+b[2])/(2*GEOM_SCALE), (b[1]+b[3])/(2*GEOM_SCALE)
    
    curr_trees = [t.clone() for t in trees]
    
    for _ in range(iterations):
        idx = random.randint(0, len(trees)-1)
        t = curr_trees[idx]
        
        # Бэкап
        ox, oy, oa = t.center_x, t.center_y, t.angle
        
        # 1. Поворот отдельного дерева (30% шанс)
        if random.random() < 0.3:
            angle_delta = random.uniform(-15, 15)
            t.angle += Decimal(angle_delta)
            t.update_polygon()
        
        # 2. Сдвиг к центру
        tx, ty = float(t.center_x), float(t.center_y)
        dx, dy = cx - tx, cy - ty
        dist = math.hypot(dx, dy)
        
        step = 0.05
        if dist > 1e-9:
            # Сдвиг + Шум
            move_x = (dx/dist)*step*random.random() + random.uniform(-0.02, 0.02)
            move_y = (dy/dist)*step*random.random() + random.uniform(-0.02, 0.02)
            t.move(move_x, move_y)
            
        # Проверка
        if not check_overlap(curr_trees):
            s = get_bounding_box_side(curr_trees)
            if s < min_side:
                # УЛУЧШЕНИЕ
                min_side = s
                best_trees = [ct.clone() for ct in curr_trees]
                # Обновляем центр масс
                b = unary_union([ct.polygon for ct in best_trees]).bounds
                cx, cy = (b[0]+b[2])/(2*GEOM_SCALE), (b[1]+b[3])/(2*GEOM_SCALE)
            elif s > min_side + Decimal('1e-9'):
                # Стало хуже -> ОТКАТ
                t.center_x, t.center_y, t.angle = ox, oy, oa
                t.update_polygon()
        else:
            # Пересечение -> ОТКАТ
            t.center_x, t.center_y, t.angle = ox, oy, oa
            t.update_polygon()
            
    return best_trees

# ==================================================================================
# 4. MAIN RUNNER
# ==================================================================================

def run_timer_optimization():
    print(f"⏱️ 10-MINUTE OPTIMIZER STARTED. Scale={GEOM_SCALE}")
    
    if FORCE_RESET or not os.path.exists(SUBMISSION_CSV):
        print("♻️ Resetting to original input...")
        copy(INPUT_CSV, SUBMISSION_CSV)
    
    best_trees = parse_csv(SUBMISSION_CSV)
    start_total = sum(get_score(t, gid) for gid, t in best_trees.items())
    print(f"🏁 STARTING SCORE: {start_total:.10f}")
    
    end_time = datetime.now() + timedelta(minutes=MAX_MINUTES)
    iter_num = 0
    
    while datetime.now() < end_time:
        iter_num += 1
        
        # --- ВЫБОР ЦЕЛЕЙ ---
        # Сортируем все группы по вкладу в скор.
        # Игнорируем N < 4 (слишком сложно улучшить).
        candidates = []
        for gid, t_list in best_trees.items():
            if int(gid) >= 4: 
                candidates.append((gid, get_score(t_list, gid)))
        
        candidates.sort(key=lambda x: x[1], reverse=True)
        
        # Берем 3 худших + 2 случайных (чтобы пробовать разные N)
        targets = [x[0] for x in candidates[:3]]
        
        all_keys = [k for k in best_trees.keys() if int(k) >= 4]
        if all_keys:
            targets.extend(random.sample(all_keys, 2))
        targets = list(set(targets))
        
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Iter {iter_num} Targets: {targets}")
        
        improved_any = False
        
        for gid in targets:
            if datetime.now() > end_time: break
            
            orig = best_trees[gid]
            orig_score = get_score(orig, gid)
            
            # 1. Сначала оптимизируем поворот всей группы
            work = optimize_rotation_group([t.clone() for t in orig])
            
            # 2. Агрессивная тряска (тут происходит магия)
            work = aggressive_shake(work, iterations=250)
            
            # 3. Финальный поворот
            work = optimize_rotation_group(work)
            
            # Проверка
            if not check_overlap(work):
                new_score = get_score(work, gid)
                if new_score < orig_score - Decimal('1e-12'):
                    print(f"  ✅ YES! N={gid}: {orig_score:.10f} -> {new_score:.10f}")
                    best_trees[gid] = work
                    improved_any = True
                    save_csv(best_trees, SUBMISSION_CSV)
        
        if improved_any:
            curr_total = sum(get_score(t, gid) for gid, t in best_trees.items())
            print(f"🔥 NEW TOTAL SCORE: {curr_total:.10f}")

    print("\n✅ 10 MINUTES UP. FINISHED.")
    make_zip()

def make_zip():
    fname = f'santa_final_{datetime.now().strftime("%H%M")}.zip'
    with ZipFile(fname, 'w', compression=ZIP_DEFLATED) as zf:
        if os.path.exists(SUBMISSION_CSV): zf.write(SUBMISSION_CSV)
    print(f"📦 Archive: {fname}")

if __name__ == "__main__":
    run_timer_optimization()

⏱️ 10-MINUTE OPTIMIZER STARTED. Scale=100000.0
🏁 STARTING SCORE: 117.8588256124

[15:31:33] Iter 1 Targets: ['020', '158', '011', '005', '024']
  ✅ YES! N=005: 0.9270787253 -> 0.8224962065
🔥 NEW TOTAL SCORE: 117.7542430936

[15:31:34] Iter 2 Targets: ['020', '014', '011', '005', '024']
  ✅ YES! N=005: 0.8224962065 -> 0.7929438861
🔥 NEW TOTAL SCORE: 117.7246907732

[15:31:36] Iter 3 Targets: ['020', '063', '025', '024', '041']

[15:31:37] Iter 4 Targets: ['020', '129', '150', '025', '024']

[15:31:38] Iter 5 Targets: ['020', '044', '084', '025', '024']

[15:31:39] Iter 6 Targets: ['020', '061', '025', '024', '135']

[15:31:40] Iter 7 Targets: ['020', '164', '025', '104', '024']

[15:31:41] Iter 8 Targets: ['020', '025', '058', '190', '024']

[15:31:42] Iter 9 Targets: ['020', '132', '165', '025', '024']

[15:31:43] Iter 10 Targets: ['020', '139', '164', '025', '024']

[15:31:45] Iter 11 Targets: ['020', '009', '025', '024', '045']

[15:31:45] Iter 12 Targets: ['090', '020', '017', '025'